# Вводные

In [1]:
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
# для человечности
from fake_useragent import UserAgent
UserAgent().chrome
from typing import List
import pandas as pd
import numpy as np
import os

from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC

In [2]:
df = pd.read_csv('kkal_bju.csv')

In [3]:
# дропнуть спортивное питание

In [11]:
df['name'].unique()

array(['Лисички свежие', 'Шампиньоны Hortex резаные',
       'Шампиньоны Бондюэль резаные', ..., 'Сарада Тори но маринадо',
       'Тори папурика', 'Кани гуриру'], dtype=object)

In [5]:
df = df[df['categories'] != 'Спортивное питание'].reset_index(drop=True)

In [14]:
# Создаем объект UserAgent и генерируем случайный Chrome user agent
user_agent = UserAgent().chrome

In [15]:
# Настройка опций Chrome для драйвера
chrome_options = Options()
#chrome_options.add_argument('--headless')  # если хотите запустить в фоновом режиме
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # рекомендуется для Linux, но может помочь и на Mac
chrome_options.add_argument(f"user-agent={user_agent}")

# Запуск Chrome с новыми настройками
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

In [16]:
class GooImage():
    def __init__(self, word):

        self.word = word
        self.url = f'https://www.google.com/search?tbm=isch&q={self.word}'
        self.random_seconds = np.arange(1.2,2.6,step=0.15)
        
        # Создаем папку, если её нет
        if not os.path.exists(f"image/{self.word}"):
            os.makedirs(f"image/{self.word}")
        

    def go_to_page(self):
        driver.get(self.url)
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        # для полной подругизки страницы
        
        # Скроллим вниз до конца текущей видимой части страницы
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Ждём загрузку контента
        time.sleep(5)

    def change_word(self, word):
        self.word = word
        self.url = f'https://www.google.com/search?tbm=isch&q={self.word}'

        # Создаем папку, если её нет
        if not os.path.exists(f"image/{self.word}"):
            os.makedirs(f"image/{self.word}")

    def save_image(self, image_url, iter):

        # Скачиваем изображение и сохраняем его
        image_data = requests.get(image_url).content
        with open(f"image/{self.word}/{self.word}_image{iter}.jpg", "wb") as file:  # Укажите имя файла
            file.write(image_data)

        return f"image/{self.word}/{self.word}_image{iter}.jpg"
        
    def scrapping_page(self, df, count_image = 20):
        #переход на стринчку
        self.go_to_page()
        
        # получение элементов (картинок)
        try:
            elems = driver.find_elements(By.CSS_SELECTOR, '.eA0Zlc.WghbWd.FnEtTd.mkpRId.m3LIae.RLdvSe.qyKxnc.ivg-i.PZPZlf.GMCzAd')
        except Exception as e:
            print(f'у ебучего {self.word} другой класс и {e}')
            return

        for iter in tqdm(range(count_image)):
            
            sec = np.random.choice(self.random_seconds)
            time.sleep(sec)
            
            try:
                # нажимаем на критинку 
                elems[iter].click()
            
                # Ждем загрузки миниатюр
                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.sFlh5c.FyHeAf.iPVvYb')))
    
                # забираем урл кратинки
                image_url = driver.find_element(By.CSS_SELECTOR, '.sFlh5c.FyHeAf.iPVvYb').get_attribute('src')
    
                # сохраняем
                desctop_url = self.save_image(image_url, iter)
    
                # записываем в список
                df.append([self.word,iter , desctop_url])
                
            except Exception as e:
                
                print(f"Произошла ебучая ошибка: {e}")


In [17]:
# создаем объект и список
df_new = []
a = GooImage(df['name'].unique()[0])
a.scrapping_page(df_new,count_image=30)

for word in tqdm(df['name'].unique()[1:]):
    a.change_word(word)
    a.scrapping_page(df_new,count_image=30)

  0%|          | 0/30 [00:00<?, ?it/s]

Произошла ебучая ошибка: Message: 

Произошла ебучая ошибка: Message: 



  0%|          | 0/3902 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Произошла ебучая ошибка: Message: 



...

Произошла ебучая ошибка: Message: 



  0%|          | 0/30 [00:00<?, ?it/s]

Произошла ебучая ошибка: Message: 

Произошла ебучая ошибка: Message: 

Произошла ебучая ошибка: Message: 



  0%|          | 0/30 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [20]:
df_new

[['Лисички свежие', 0, 'image/Лисички свежие/Лисички свежие_image0.jpg'],
 ['Лисички свежие', 1, 'image/Лисички свежие/Лисички свежие_image1.jpg'],
 ['Лисички свежие', 2, 'image/Лисички свежие/Лисички свежие_image2.jpg'],
 ['Лисички свежие', 3, 'image/Лисички свежие/Лисички свежие_image3.jpg'],
 ['Лисички свежие', 4, 'image/Лисички свежие/Лисички свежие_image4.jpg'],
 ['Лисички свежие', 5, 'image/Лисички свежие/Лисички свежие_image5.jpg'],
 ['Лисички свежие', 6, 'image/Лисички свежие/Лисички свежие_image6.jpg'],
 ['Лисички свежие', 7, 'image/Лисички свежие/Лисички свежие_image7.jpg'],
 ['Лисички свежие', 8, 'image/Лисички свежие/Лисички свежие_image8.jpg'],
 ['Лисички свежие', 9, 'image/Лисички свежие/Лисички свежие_image9.jpg'],
 ['Лисички свежие', 10, 'image/Лисички свежие/Лисички свежие_image10.jpg'],
 ['Лисички свежие', 11, 'image/Лисички свежие/Лисички свежие_image11.jpg'],
 ['Лисички свежие', 12, 'image/Лисички свежие/Лисички свежие_image12.jpg'],
 ['Лисички свежие', 13, 'image/Л

In [21]:
total = pd.DataFrame(df_new, columns = ['name','id','url'])

In [27]:
total.to_csv('under_Йогурт Чудо Двойной вкус Клубника-Земляника.csv', index=False)

In [26]:
total.iloc[-1,-1]

'image/Йогурт Чудо Двойной вкус Клубника-Земляника/Йогурт Чудо Двойной вкус Клубника-Земляника_image29.jpg'